In [1]:
import numpy as np
import dateutil.parser
import datetime
from urllib.request import urlopen, Request
import simplejson as json
import pandas as pd

In [2]:
tomorrow = datetime.datetime.today()+ datetime.timedelta(days=1)
time = tomorrow.strftime('%Y-%m-%d') + "T00:00:00"
with open('apikey') as f:
    apikey = f.read().strip()
num_ens = 10
prec_var = "Total_precipitation_surface_6_Hour_Accumulation_ens"
pres_var = "Pressure_surface_ens"

In [7]:
API_meta_url = "http://api.planetos.com/v1/datasets/noaa-ncep_gefs?apikey={}".format(apikey)
print(API_meta_url)
request = Request(API_meta_url)
response = urlopen(request)
API_meta = json.loads(response.read())

[i['name'] for i in API_meta['Variables'] if 'precipitation' in i['name'].lower() and 'surface' in i['name'].lower()]

http://api.planetos.com/v1/datasets/noaa-ncep_gefs?apikey=d234affcd90a44538749b23fcab5dcd8


HTTPError: HTTP Error 503: Service Unavailable: Back-end server is at capacity

In [ ]:
API_url = "http://api.planetos.com/v1/datasets/noaa-ncep_gefs/point?time={}&count=100000&verbose=false&apikey={}&var={}".format(time, apikey, prec_var)
request = Request(API_url)
response = urlopen(request)
API_data_prec = json.loads(response.read())
print(API_url)

In [4]:
lats = []
lons = []
for i in API_data_prec['entries']:
    lats.append(i['axes']['latitude'])
    lons.append(i['axes']['longitude'])

lats = np.unique(lats)
lons = np.unique(lons)

NameError: name 'API_data_prec' is not defined

In [ ]:
## first collect data to dictionaries, then convert to Pandas DataFrame

X = np.zeros((len(lons),len(lats)), 2)
Y = np.zeros((len(lons), len(lats), num_ens))

min_lat = int(np.min(lats))
min_lon = int(np.min(lons))
for i in API_data_prec['entries']:
    lat = int(i['axes']['latitude'])
    lon = int(i['axes']['longitude'])
    ens = int(i['axes']['ens'])
    X[lon-min_lon, lat-min_lat, 0] = lat
    X[lon-min_lon, lat-min_lat, 1] = lon
    Y[lon-min_lon, lat-min_lat, ens] = i['data'][prec_var]

np.save("latlon_{}.npy".format(time), X)
np.save("precipitation_{}.npy".format(time), Y)